In [243]:
from mtspHeuristic import *
from mtspLocalSearch import *
from mtspConstruction import *
import random

In [244]:
def localSearchProcedure(solution, obj_function, distance_matrix, number_of_iterations = 10):
    iterations = 0
    while iterations < number_of_iterations:
        solution, obj_function = local_search_swap_interroute(solution, obj_function, distance_matrix)
        solution, obj_function = local_search_shift_interroute(solution, obj_function, distance_matrix)
        solution, obj_function = local_search_swap_intraroute(solution, obj_function, distance_matrix)
        solution, obj_function = local_search_shift_intraroute(solution, obj_function, distance_matrix)
        iterations += 1
    return solution, obj_function

In [245]:
test_graph = [[0,8,5,10,6,3,13,2,10,6,3,4,9,6,17],
         [8,0,9,6,12,7,4,9,15,9,10,6,5,9,11],
         [5,9,0,10,9,9,7,18,7,4,7,3,11,4,10],
         [10,6,10,0,18,2,10,6,5,4,9,12,8,8,10],
         [6,12,9,18,0,9,10,11,11,12,6,5,6,8,9],
         [3,7,9,2,9,0,6,8,9,10,5,2,7,10,6],
         [13,4,7,10,10,6,0,12,8,3,7,4,10,6,9],
         [2,9,18,6,11,8,12,0,7,3,6,18,17,9,2],
         [10,15,7,5,11,9,8,7,0,10,8,3,14,10,3],
         [6,9,4,4,12,10,3,3,10,0,12,5,9,6,3],
         [3,10,7,9,6,5,7,6,8,12,0,19,8,8,11],
         [4,6,3,12,5,2,4,18,3,5,19,0,12,2,9],
         [9,5,11,8,6,7,10,17,14,9,8,12,0,5,6],
         [6,9,4,8,8,10,6,9,10,6,8,2,5,0,2],
         [17,11,10,10,9,6,9,2,3,3,11,9,6,2,0]]

number_of_salesmen = 3
solution, obj_function = greedy_insertion_construction(test_graph, number_of_salesmen)

#### Testing Disturbance Procedure Function

In [246]:
def disturbanceProcedure(solution, obj_function, distance_matrix):
    solution, obj_function = generate_random_neighbor_shift_interroute(solution, obj_function, distance_matrix)
    solution, obj_function = generate_random_neighbor_swap_interroute(solution, obj_function, distance_matrix)
    solution, obj_function = generate_random_neighbor_shift_intraroute(solution, obj_function, distance_matrix)
    solution, obj_function = generate_random_neighbor_swap_intraroute(solution, obj_function, distance_matrix)
    return solution, obj_function

In [247]:
a, b = disturbanceProcedure(solution, obj_function, test_graph)
print(b)
print(solution, obj_function, objective_function_vector_bf(solution,test_graph), sum(obj_function))
a, b, objective_function_vector_bf(a, test_graph), sum(b)

[44, 47, 23]
[[10, 4, 9, 12, 3], [6, 8, 2, 13, 15, 14, 5], [7, 11]] [26, 47, 23] [26, 47, 23] 96


([[4, 12, 9, 10, 3], [6, 8, 2, 13, 15, 14, 5], [7, 11]],
 [44, 47, 23],
 [44, 47, 23],
 114)

### Variable Neighborhood Descent

In [248]:
def variableNeighborhoodDescent(solution, obj_function, distance_matrix, num_iterations = 1000):
    k = 0
    iterations = 0
    solution_current = solution.copy()
    obj_function_vector_current = obj_function.copy()
    obj_function_current = objective_function_minsum(obj_function)
    best_solution = solution.copy()
    best_OF_vector = obj_function.copy()
    best_OF = objective_function_minsum(best_OF_vector)
    
    #Etapa de busca em vizinhanças variáveis
    while iterations < num_iterations and k < 5:
        if k == 0:
            neighbor_sol, neighbor_OF = local_search_swap_intraroute(solution_current,obj_function_vector_current,distance_matrix)
        if k == 1:
            neighbor_sol, neighbor_OF = local_search_two_opt(solution_current,obj_function_vector_current,distance_matrix)
        if k == 2:
            neighbor_sol, neighbor_OF = local_search_shift_intraroute(solution_current,obj_function_vector_current,distance_matrix)
        if k == 3:
            neighbor_sol, neighbor_OF = local_search_swap_interroute(solution_current,obj_function_vector_current,distance_matrix)
        if k == 4:
            neighbor_sol, neighbor_OF = local_search_shift_interroute(solution_current,obj_function_vector_current,distance_matrix) 
        
        #Etapa de avaliação da solução
        if objective_function_minsum(neighbor_OF) < obj_function_current:
            solution_current = neighbor_sol.copy()
            obj_function_vector_current = neighbor_OF.copy()
            obj_function_current = objective_function_minsum(obj_function_vector_current)

            if objective_function_minsum(neighbor_OF) < best_OF:
                best_OF_vector = obj_function_vector_current.copy()
                best_solution = solution_current.copy()
                best_OF = objective_function_minsum(best_OF_vector)
                #print("found better solution with OF calc", best_OF, "and OF True", objective_function_vector_bf(best_solution,distance_matrix))
            k = 0
        else: # Não houve melhoria
            k += 1
        #print("k = %d at iteration %d" % (k, iterations))
        iterations += 1
    return best_solution, best_OF_vector

In [249]:
a, b = variableNeighborhoodDescent(solution, obj_function, test_graph)
print(b)
print(solution, obj_function, objective_function_vector_bf(solution,test_graph))
a, b, objective_function_vector_bf(a, test_graph), sum(b)

[34, 27, 6]
[[10, 4, 9, 12, 3], [6, 8, 2, 13, 15, 14, 5], [7, 11]] [26, 47, 23] [26, 47, 23]


([[5, 12, 9, 7, 10, 3], [8, 15, 14, 13, 2, 4, 6], [11]],
 [34, 27, 6],
 [34, 27, 6],
 67)

### Variable Neighborhood Search <br>
(com reinício em perturbação de soluções correntes) <br> (with disturbance procedure on current solutions as restart) 

In [250]:
def variableNeighborhoodSearch(solution, obj_function, distance_matrix,num_iterations = 1000):
    k = 0
    iterations = 0
    solution_current = solution.copy()
    obj_function_vector_current = obj_function.copy()
    obj_function_current = objective_function_minsum(obj_function_vector_current)
    flag_have_better_solutions = False
    best_solution = solution.copy()
    best_OF_vector = obj_function.copy()
    best_OF = objective_function_minsum(best_OF_vector)
    
    #Etapa de geração de vizinhanças
    while iterations < num_iterations:
        if flag_have_better_solutions == False:
            if k == 0:
                neighbor_sol, neighbor_OF = generate_random_neighbor_swap_intraroute(solution_current,obj_function_vector_current,distance_matrix)
            if k == 1:
                neighbor_sol, neighbor_OF = generate_random_neighbor_two_opt(solution_current,obj_function_vector_current,distance_matrix)
            if k == 2:
                neighbor_sol, neighbor_OF = generate_random_neighbor_shift_intraroute(solution_current,obj_function_vector_current,distance_matrix)
            if k == 3:
                neighbor_sol, neighbor_OF = generate_random_neighbor_swap_interroute(solution_current,obj_function_vector_current,distance_matrix)
            if k == 4:
                neighbor_sol, neighbor_OF = generate_random_neighbor_shift_interroute(solution_current,obj_function_vector_current,distance_matrix) 
        
        #Etapa de busca
        neighbor_sol, neighbor_OF = localSearchProcedure(solution, obj_function, distance_matrix)

        if objective_function_minsum(neighbor_OF) < obj_function_current:
            solution_current = neighbor_sol.copy()
            obj_function_vector_current = neighbor_OF.copy()
            obj_function_current = objective_function_minsum(obj_function_vector_current)

            if objective_function_minsum(neighbor_OF) < best_OF:
                best_OF_vector = obj_function_vector_current.copy()
                best_solution = solution_current.copy()
                best_OF = objective_function_minsum(best_OF_vector)
                #print("found better solution with OF calc", best_OF, "and OF True", objective_function_vector_bf(best_solution,distance_matrix))
            flag_have_better_solutions = True

            k = 0
        else:
            flag_have_better_solutions = False
            k += 1

        if k == 5: # Etapa de reinício aleatório
            k = 0
            solution_current, obj_function_vector_current = disturbanceProcedure(solution_current, obj_function_vector_current, distance_matrix)
            obj_function_current = objective_function_minsum(obj_function_vector_current)

        iterations += 1
    return best_solution, best_OF_vector

In [251]:
a, b = variableNeighborhoodSearch(solution, obj_function, test_graph)
print(b)
print(solution, obj_function, objective_function_vector_bf(solution,test_graph))
a, b, objective_function_vector_bf(a, test_graph), sum(b)

[17, 31, 20]
[[10, 4, 9, 12, 3], [6, 8, 2, 13, 15, 14, 5], [7, 11]] [26, 47, 23] [26, 47, 23]


([[6, 4, 9, 12], [8, 15, 10, 7, 2, 13, 5], [14, 3, 11]],
 [17, 31, 20],
 [17, 31, 20],
 68)

### Iterated Local Search

In [252]:
def iteratedLocalSearch(solution, obj_function, distance_matrix, num_iterations = 2000):
    k = 0
    iterations = 0
    solution_current = solution.copy()
    obj_function_vector_current = obj_function.copy()
    obj_function_current = objective_function_minsum(obj_function_vector_current)
    best_solution = solution.copy()
    best_OF_vector = obj_function.copy()
    best_OF = objective_function_minsum(best_OF_vector)
    while(iterations < num_iterations):
        neighbor_sol, neighbor_OF = localSearchProcedure(solution_current, obj_function_vector_current, distance_matrix)
        if objective_function_minsum(neighbor_OF) < obj_function_current:
            solution_current = neighbor_sol.copy()
            obj_function_vector_current = neighbor_OF.copy()
            obj_function_current = objective_function_minsum(obj_function_vector_current)
            
            if obj_function_current < best_OF:
                best_OF_vector = obj_function_vector_current.copy()
                best_solution = solution_current.copy()
                best_OF = obj_function_current
            #Perturbação da solução
            solution, obj_function = disturbanceProcedure(solution, obj_function,distance_matrix)
        iterations += 1

    return best_solution, best_OF_vector

In [253]:
a, b = iteratedLocalSearch(solution, obj_function, test_graph)
print(b)
print(solution, obj_function, objective_function_vector_bf(solution,test_graph))
a, b, objective_function_vector_bf(a, test_graph), sum(b)

[17, 31, 20]
[[7, 4, 9, 3], [12, 6, 8, 2, 13, 15, 14, 5], [10, 11]] [40, 50, 21] [40, 50, 21]


([[6, 4, 9, 12], [8, 15, 10, 7, 2, 13, 5], [14, 3, 11]],
 [17, 31, 20],
 [17, 31, 20],
 68)